# DSPy Tutorial: Building a Code Plagiarism Detector
Based on
https://github.com/stanfordnlp/dspy/blob/main/examples/outdated_v2.4_examples/multi-input-output/beginner-multi-input-output.ipynb

In [12]:
import os
import re

import dspy
import pandas as pd
from dotenv import load_dotenv
from dspy.evaluate import Evaluate
from dspy.teleprompt import (
    BootstrapFewShot,
    BootstrapFewShotWithRandomSearch,
    KNNFewShot,
    MIPROv2,
)

# load your environment variables from .env file
load_dotenv()

# openai model deployment
OPENAI_MODEL = os.getenv("OPENAI_MODEL")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

JUDGE_MODEL = OPENAI_MODEL
JUDGE_API_KEY = OPENAI_API_KEY

print(f"Using OpenAI model: {OPENAI_MODEL}")
print(f"Using OpenAI API key: {OPENAI_API_KEY[:3]}...{OPENAI_API_KEY[-2:]}")

print(f"Using Judge model: {JUDGE_MODEL}")
print(f"Using Judge API key: {JUDGE_API_KEY[:3]}...{JUDGE_API_KEY[-2:]}")

Using OpenAI model: openai/gpt-4o-mini
Using OpenAI API key: sk-...sA
Using Judge model: openai/gpt-4o-mini
Using Judge API key: sk-...sA


# 0. Load dataset
Our first task is to load our dataset. Each entry in our dataset will consist of the following components:

* sample_1: The first code sample to be analyzed
* sample_2: The second code sample to be compared against the first
* plagiarized: A boolean value (True if plagiarism is detected, False otherwise)
* reason: A detailed explanation of the plagiarism detection result

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/williambrach/LLM-plagiarism-check/main/data/train.tsv", sep="\t")
df.head()

,L,case,sample_1,sample_2,plagiarized,reason
0,0,1,public class T1 { public static void main(Str...,"/* * To change this license header, choose Li...",False,"The two code samples, while producing similar ..."
1,0,1,public class T1 { public static void main(Str...,/** * * @author 65FBEF05E01FAC390CB3FA073FB3...,False,The code samples demonstrate different approac...
2,0,1,public class T1 { public static void main(Str...,/** * * @author CB6AB3315634A1E4D11B091BA48B...,False,The two code samples produce the same output b...
3,1,1,public class T1 { public static void main(Str...,"* * To change this license header, choose Lice...",True,The two code samples are nearly identical in t...
4,2,1,public class T1 { public static void main(Str...,"/* * To change this license header, choose Li...",True,The two code samples contain identical main me...


# 1. Prepare dataset
DSPy utilizes special objects called Example to structure and process data. Our next task is to convert our raw dataset into a collection of these Example objects.

## Creating Custom Example Objects
For our plagiarism detection system, we'll create Example objects with the following attributes:

* code_sample_1: The first code sample to analyze
* code_sample_2: The second code sample to compare
* plagiarized: Boolean indicating whether plagiarism was detected
* explanation: Detailed reasoning for the plagiarism decision

## Specifying Inputs
It's crucial to inform DSPy which attributes serve as inputs for our model. We accomplish this using the `.with_inputs()` method. In our case, we'll specify:

`.with_inputs("code_sample_1", "code_sample_2")`

In [3]:
def create_example(row: pd.Series) -> dspy.Example:
    return dspy.Example(
        code_sample_1=row["sample_1"],
        code_sample_2=row["sample_2"],
        plagiarized="Yes" if row["plagiarized"] else "No",
        explanation=row["reason"],
    ).with_inputs("code_sample_1", "code_sample_2")


examples = []
for _, row in df.iterrows():
    example = create_example(row)
    examples.append(example)

# 2. DSPy setup
Experiments with OpenAI model

In [4]:
lm = dspy.LM(
    api_key=OPENAI_API_KEY,
    model=OPENAI_MODEL,
)
dspy.settings.configure(lm=lm)

# 3. Setting up signature and module
## Core DSPy Components
dspy.Signature and dspy.Module are fundamental building blocks for DSPy programs:

* Signature: A declarative specification of the input/output behavior of a DSPy module.
* Module: A building block for programs that leverage Language Models (LMs).

## Types of DSPy Modules

DSPy offers various module types, each serving different purposes:

1.  **`dspy.Predict`**

    *   Basic predictor
    *   Maintains the original signature
    *   Handles key forms of learning (storing instructions, demonstrations, and LM updates)
    *   Most similar to direct LM usage

2.  **`dspy.ChainOfThought`**

    *   Enhances the LM to think step-by-step before producing the final response
    *   Modifies the signature to incorporate intermediate reasoning steps

3.  **Additional Advanced Modules**

    *   DSPy library offers a range of more specialized modules for complex tasks

## Recommendation for starting
For those new to DSPy, it's advisable to start with dspy.Predict. Its simplicity makes it ideal for understanding the basics of DSPy operation. Once you've successfully implemented your program using dspy.Predict, you can explore more advanced modules like dspy.ChainOfThought to potentially enhance your model's performance.

For an overview of other prompting techniques beyond zero-shot learning, refer to the Prompting Guide. This resource covers various methods that can enhance your DSPy applications as you progress.

In [5]:
class PlagiarismSignature(dspy.Signature):
    # Clarify something about the nature of the task (expressed below as a docstring)!
    """Detect if two code samples are plagiarized. In plagiarized field answer only : Yes if the code samples are plagiarized, No otherwise. In explanation field add the reason why the code samples are/ are not plagiarized."""

    # Supply hints on the nature of an input field, expressed as a desc keyword argument for dspy.InputField.
    code_sample_1 = dspy.InputField(desc="The first code sample to compare")
    code_sample_2 = dspy.InputField(desc="The second code sample to compare")

    # Supply constraints on an output field, expressed as a desc keyword argument for dspy.OutputField.
    explanation = dspy.OutputField(
        desc="Explanation or reason why the code samples are/ are not plagiarized"
    )
    plagiarized = dspy.OutputField(
        desc="Yes/No indicating if code samples are plagiarized"
    )


class PlagiarismCoT(dspy.Module):
    def __init__(self) -> None:
        super().__init__()

        # self.prog = dspy.ChainOfThought(PlagiarismSignature)
        self.prog = dspy.Predict(PlagiarismSignature)

    def forward(self, code_sample_1: str, code_sample_2: str) -> PlagiarismSignature:

        # here you can do any processing you want, calling your function, etc.
        # modifying your code, inputs etc.
        # similar to pytorch forward function

        # returned signature object
        prediction = self.prog(code_sample_1=code_sample_1, code_sample_2=code_sample_2)
        return prediction

# 3.1 Test your module with zero-shot program

In [7]:
print(examples[0].code_sample_1)
print(examples[0].code_sample_1)

public class T1 {  public static void main(String[] args) {  System.out.println("Welcome to Java");  System.out.println("Welcome to Java");  System.out.println("Welcome to Java");  System.out.println("Welcome to Java");  System.out.println("Welcome to Java");  }  }
public class T1 {  public static void main(String[] args) {  System.out.println("Welcome to Java");  System.out.println("Welcome to Java");  System.out.println("Welcome to Java");  System.out.println("Welcome to Java");  System.out.println("Welcome to Java");  }  }


In [6]:
output = PlagiarismCoT()(examples[0].code_sample_1, examples[0].code_sample_1)
print(f"plagiarized : {output.plagiarized}")
print(f"explanation : {output.explanation}")

plagiarized : Yes
explanation : The two code samples are identical, with the same class name, method, and repeated print statements. This indicates that one code sample is likely copied from the other without any modification, which constitutes plagiarism.


# 4. Create metric for your programs

Creating evaluation metric is a crucial component of the DSPy pipeline. This step is often the most complex function in your DSPy application.

## Metric Structure

In DSPy, metrics (or evaluation functions) must adhere to the following structure:

1.  **Arguments:**

    *   Two required arguments, both of type `dspy.Example`:
        *   The ground truth example from the dataset
        *   The predicted example from the model
    *   An optional third argument called `trace`

2.  **Return Value:**

    *   A numerical score (`float`, `int`, or `bool`)
    *   For binary metrics, return `True` for correct predictions and `False` for incorrect ones

For more detailed information on creating metrics in DSPy, refer to the [official documentation on metrics](https://dspy-docs.vercel.app/docs/building-blocks/metrics).

In [8]:
def validate_answer(
    example: dspy.Example, pred: PlagiarismSignature, trace: object = None
) -> bool:
    """
    Validate the predicted plagiarism answer against the example answer.

    This function compares the predicted plagiarism answer with the example answer,
    focusing on a simple "yes" or "no" response. It extracts the core answer from
    the prediction, handling potential variations in formatting and capitalization.

    Parameters:
    - example (dspy.Example): The example object containing the correct answer.
    - pred (PlagiarismSignature): The prediction object containing the model's answer.
    - trace (object, optional): Unused parameter, kept for compatibility.

    Returns:
    - bool: True if the predicted answer matches the example answer, False otherwise.

    The function returns False if either the predicted or example answer is None,
    or if any exception occurs during the validation process.
    """
    try:
        if pred.plagiarized is None:
            return False

        # Extract the first line of the predicted answer, convert to lowercase
        pred_plag = pred.plagiarized.strip().lower().split("\n")[0]

        # Define a regex pattern to match "yes" or "no"
        yes_no_pattern = r"\b(yes|no)\b"

        # Search for the pattern in the predicted answer
        match = re.search(yes_no_pattern, pred_plag)

        # If a match is found, use it; otherwise, use the entire predicted answer
        extracted_answer = match.group(1) if match else pred.plagiarized.strip().lower()

        if example.plagiarized is None:
            return False

        score = (
            True if extracted_answer == example.plagiarized.strip().lower() else False
        )
    except Exception:
        score = False
    return score

# 4.1 Test your metric with evaluate
Now we can evaluate zero-shot program with our `validate_answer` function. This will be bottom baseline for our program.

In [9]:
evaluate = Evaluate(
    devset=examples,
    metric=validate_answer,
    num_threads=4,
    display_progress=True,
    display_table=False,
)

# zero-shot evaluation on examples
score = evaluate(PlagiarismCoT())

Average Metric: 28.00 / 63 (44.4%): 100%|██████████| 63/63 [00:39<00:00,  1.59it/s]

2025/03/14 17:36:49 INFO dspy.evaluate.evaluate: Average Metric: 28 / 63 (44.4%)


In [10]:
# score for zero-shot evaluation with `validate_answer` function
print(f"Zero-shot score: {score}")

Zero-shot score: 44.44


# 4.2 When you can't use simple programmatic metrics and you need more brain power
In our metric, we are only evaluating the plagiarized field. We are not evaluating the explanation field. If we wanted to evaluate the explanation, which is a string (reasoning), we could use a text similarity metric (cosine similarity, for example) or another LLM to evaluate it.

The advantage of using another LLM is that we can evaluate the explanation in a more human-like way, but be careful with this approach. It could be very expensive and add more time to your program's runtime.

More examples of implementing this type of metric can be found in the [tweets example](https://github.com/stanfordnlp/dspy/blob/main/examples/tweets/tweet_metric.py).

In [13]:
# Define the signature for automatic assessments.
class Assess(dspy.Signature):
    """Assess the similarity of a of inputs. Answer only Yes if inputs are similar or No if not."""

    original_explanation = dspy.InputField()
    predicted_reasoning = dspy.InputField()
    similar = dspy.OutputField(desc="Yes if inputs are similar or No if not")


gpt_judge = dspy.LM(
    model=JUDGE_MODEL,
    api_key=JUDGE_API_KEY,
)


def validate_answer_with_explanation(
    example: dspy.Example, pred: PlagiarismSignature, trace: object = None
) -> bool:

    # Extract the true explanation from the example
    true_explanation = example.explanation

    # Extract the predicted explanation from the prediction
    pred_explanation = pred.explanation

    # Use a language model (gpt_judge) to assess the similarity of explanations
    with dspy.context(lm=gpt_judge):
        # Create an Assess object to compare the explanations
        similar = dspy.Predict(Assess)(
            original_explanation=true_explanation, predicted_reasoning=pred_explanation
        )

        # Check if the explanations are deemed similar (converting to lowercase for case-insensitive comparison)
        similar_score = similar.similar.lower() == "yes"

    # Validate the plagiarism answer using the existing validate_answer function
    plagiarized_score = validate_answer(example, pred, trace)

    # Return True only if both the explanation is similar and the plagiarism answer is correct
    return similar_score and plagiarized_score

In [14]:
evaluate_with_explanation = Evaluate(
    devset=examples,
    metric=validate_answer_with_explanation,
    num_threads=4,
    display_progress=True,
    display_table=False,
)

# zero-shot evaluation on examples
score = evaluate_with_explanation(PlagiarismCoT())

Average Metric: 25.00 / 63 (39.7%): 100%|██████████| 63/63 [00:16<00:00,  3.93it/s]

2025/03/14 17:40:53 INFO dspy.evaluate.evaluate: Average Metric: 25 / 63 (39.7%)


In [15]:
# score for zero-shot evaluation with `validate_answer_with_explanation` function
print(f"Zero-shot score for validate_answer_with_explanation : {score}")

Zero-shot score for validate_answer_with_explanation : 39.68


# 5. DSPy Magic - Optimizers

## TLDR on Optimizers

Optimizers in DSPy are powerful tools that handle prompt engineering for you. Think of them as "functions" that automate and improve the process of crafting effective prompts. There are several types of optimizers available in DSPy, but in this tutorial, we'll focus on the first two:

1.  **Automatic Few-Shot Learning:**

    *   Adds examples to your prompts.
    *   Key optimizers to explore: `BootstrapFewShot` and `BootstrapFewShotWithRandomSearch`

2.  **Automatic Instruction Optimization:**

    *   Produces optimal instructions for the prompt.
    *   In the case of MIPRO, also optimizes the set of few-shot examples.
    *   Key optimizer to explore: `MIPROv2`

3.  Automatic Finetuning:

    *   Used to fine-tune the underlying Language Model(s).

4.  Program Transformations:

    * Ensembles a set of DSPy programs and either uses the full set or randomly samples a subset into a single program.

If you're new to optimizers and unsure where to start, `BootstrapFewShotWithRandomSearch` is a safe and effective choice for beginners.

## Read the Documentation!

For more detailed information about optimizers, please refer to the [official DSPy documentation on optimizers](https://dspy-docs.vercel.app/docs/building-blocks/optimizers).

## 5.1 Example of BootstrapFewShot

[BootstrapFewShot docs](https://dspy-docs.vercel.app/docs/deep-dive/teleprompter/bootstrap-fewshot), [BootstrapFewShot class](https://github.com/stanfordnlp/dspy/blob/main/dspy/teleprompt/bootstrap.py)

In [16]:
config = {"max_bootstrapped_demos": 4, "max_labeled_demos": 4}

optimizer = BootstrapFewShot(metric=validate_answer, **config)
optimized_program = optimizer.compile(PlagiarismCoT(), trainset=examples)
score = evaluate(optimized_program)
optimized_program.save("BootstrapFewShot_program.json")

  8%|▊         | 5/63 [00:08<01:35,  1.65s/it]


Bootstrapped 4 full traces after 5 examples for up to 1 rounds, amounting to 5 attempts.
Average Metric: 32.00 / 63 (50.8%): 100%|██████████| 63/63 [00:40<00:00,  1.57it/s]

2025/03/14 17:44:30 INFO dspy.evaluate.evaluate: Average Metric: 32 / 63 (50.8%)


In [17]:
# score for BootstrapFewShot evaluation with `validate_answer` function
print(f"BootstrapFewShot score : {score}")

BootstrapFewShot score : 50.79


# 5.2 Example of BootstrapFewShotWithRandomSearch
[BootstrapFewShotWithRandomSearch class](https://github.com/stanfordnlp/dspy/blob/main/dspy/teleprompt/random_search.py)

In [18]:
config = {
    "max_bootstrapped_demos": 4,
    "max_labeled_demos": 4,
    "num_candidate_programs": 5,
    "num_threads": 4,
}

optimizer = BootstrapFewShotWithRandomSearch(metric=validate_answer, **config)
optimized_program = optimizer.compile(PlagiarismCoT(), trainset=examples)
score = evaluate(optimized_program)
optimized_program.save("BootstrapFewShotWithRandomSearch_program.json")

Going to sample between 1 and 4 traces per predictor.
Will attempt to bootstrap 5 candidate sets.
Average Metric: 28.00 / 63 (44.4%): 100%|██████████| 63/63 [00:00<00:00, 994.53it/s] 

2025/03/14 17:50:25 INFO dspy.evaluate.evaluate: Average Metric: 28 / 63 (44.4%)



New best score: 44.44 for seed -3
Scores so far: [44.44]
Best score so far: 44.44
Average Metric: 29.00 / 63 (46.0%): 100%|██████████| 63/63 [00:24<00:00,  2.62it/s]

2025/03/14 17:50:49 INFO dspy.evaluate.evaluate: Average Metric: 29 / 63 (46.0%)



New best score: 46.03 for seed -2
Scores so far: [44.44, 46.03]
Best score so far: 46.03


...

2025/03/14 17:54:07 ERROR dspy.utils.parallelizer: Error for Example({'code_sample_1': 'public class T6 {  public static void main(String[] args) {   java.util.Scanner input = new java.util.Scanner(System.in);   int[] num = new int[10];   for (int i = 0; i < 10; i++) {    // Read a number    System.out.print("Read a number: ");    num[i] = input.nextInt();   }   // Display the array   for (int i = 9; i >= 0; i--) {    System.out.println(num[i]);   }  } }', 'code_sample_2': 'import java.util.Scanner; /**  *  * @author CB6AB3315634A1E4D11B091BA48B60BA  */ public class Nomor6_2451041557A22145B3701B0184109CAB013 { /**  * @param args the command line arguments  */ public static void main(String[] args) { Scanner sc = new Scanner(System.in); int Array[] = new int[10]; for(int i = 0 ; i<10;i++){ System.out.print("Read a number: "); Array[i] = sc.nextInt(); } for(int i = 9 ; i >= 0 ; i--){ System.out.println(Array[i]); } } }', 'plagiarized': 'No', 'explanation': "These code samples are not pla

Average Metric: 29.00 / 62 (46.8%): 100%|██████████| 63/63 [00:52<00:00,  1.19it/s]

2025/03/14 17:54:18 INFO dspy.evaluate.evaluate: Average Metric: 29.0 / 63 (46.0%)



Scores so far: [44.44, 46.03, 50.79, 74.6, 55.56, 73.02, 33.33, 46.03]
Best score so far: 74.6
8 candidate programs found.
Average Metric: 47.00 / 63 (74.6%): 100%|██████████| 63/63 [00:00<00:00, 687.69it/s]

2025/03/14 17:54:18 INFO dspy.evaluate.evaluate: Average Metric: 47 / 63 (74.6%)


In [19]:
# score for BootstrapFewShotWithRandomSearch evaluation with `validate_answer` function
print(f"BootstrapFewShotWithRandomSearch score : {score}")

BootstrapFewShotWithRandomSearch score : 74.6


# 5.3 Example of KNNFewShot
[KNNFewShot class](https://github.com/stanfordnlp/dspy/blob/main/dspy/teleprompt/knn_fewshot.py)


In [20]:
k = 3
optimizer = KNNFewShot(k, examples, vectorizer=dspy.Embedder(SentenceTransformer("all-MiniLM-L6-v2").encode))
optimized_program = optimizer.compile(PlagiarismCoT(), trainset=examples)
score = evaluate(optimized_program)
optimized_program.save("KNNFewShot_program.json")

TypeError: KNNFewShot.__init__() missing 1 required positional argument: 'vectorizer'

In [ ]:
# score for KNNFewShot evaluation with `validate_answer` function
print(f"KNNFewShot score : {score}")

# 5.4 Example of MIPROv2
[MIPROv2 paper](https://arxiv.org/abs/2406.11695), [MIPROv2 class](https://github.com/stanfordnlp/dspy/blob/main/dspy/teleprompt/mipro_optimizer_v2.py)

In [23]:
n = 5  # The number of instructions and fewshot examples that we will generate and optimize over
batches = 20  # The number of optimization trials to be run (we will test out a new combination of instructions and fewshot examples in each trial)
temperature = 1  # The temperature configured for generating new instructions
eval_kwargs = {"num_threads": 4, "display_progress": False, "display_table": 0}
optimizer = MIPROv2(
    prompt_model=lm,
    task_model=lm,
    metric=validate_answer,
    num_candidates=n,
    init_temperature=temperature,
    verbose=True,
)
optimized_program = optimizer.compile(
    PlagiarismCoT(),
    trainset=examples,
    num_trials=batches,
    max_bootstrapped_demos=16,
    max_labeled_demos=16,
    requires_permission_to_run=False,
    # eval_kwargs=eval_kwargs,
)
score = evaluate(optimized_program)
optimized_program.save("MIPROv2_program.json")

2025/03/14 18:03:57 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/03/14 18:03:57 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/03/14 18:03:57 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=5 sets of demonstrations...


Bootstrapping set 1/5
Bootstrapping set 2/5
Bootstrapping set 3/5


100%|██████████| 13/13 [01:00<00:00,  4.64s/it]


Bootstrapped 9 full traces after 12 examples for up to 1 rounds, amounting to 13 attempts.
Bootstrapping set 4/5


 23%|██▎       | 3/13 [00:05<00:19,  1.94s/it]


...

2025/03/14 18:12:52 ERROR dspy.utils.parallelizer: Error for Example({'code_sample_1': 'import java.util.Scanner; public class T7 {  public static void main(String[] args) {   Scanner input = new Scanner(System.in);   System.out.print("Enter a 4 by 4 matrix row by row: ");   double[][] m = new double[4][4];   for (int i = 0; i < 4; i++)    for (int j = 0; j < 4; j++)     m[i][j] = input.nextDouble();   System.out.print("Sum of the elements in the major diagonal is " + sumMajorDiagonal(m));  }  public static double sumMajorDiagonal(double[][] m) {   double sum = 0;   for (int i = 0; i < m.length; i++)    sum += m[i][i];   return sum;  } }', 'code_sample_2': 'import java.util.*; public class L1 { public static void main(String[] args) { Scanner input = new Scanner(System.in); System.out.print("Enter a 4 by 4 matrix row by row: "); double[][] m = new double[4][4]; for (int i = 0; i < 4; i++)  { for (int j = 0; j < 4; j++) { m[i][j] = input.nextDouble(); } } System.out.print("Sum of the el

Average Metric: 24.00 / 46 (52.2%): 100%|██████████| 50/50 [00:33<00:00,  1.51it/s]

2025/03/14 18:12:56 INFO dspy.evaluate.evaluate: Average Metric: 24.0 / 50 (48.0%)
2025/03/14 18:12:56 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [40.0, 0.0, 54.0, 48.0]
2025/03/14 18:12:56 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 54.0
2025/03/14 18:12:56 INFO dspy.teleprompt.mipro_optimizer_v2: =======================
2025/03/14 18:12:56 INFO dspy.teleprompt.mipro_optimizer_v2: 

2025/03/14 18:12:56 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 54.0!



Average Metric: 33.00 / 63 (52.4%): 100%|██████████| 63/63 [00:08<00:00,  7.52it/s]

2025/03/14 18:13:05 INFO dspy.evaluate.evaluate: Average Metric: 33 / 63 (52.4%)


In [24]:
# score for MIPROv2 evaluation with `validate_answer` function
print(f"MIPROv2 score : {score}")

MIPROv2 score : 52.38


# 6. Loading a Saved Program
Loading a program from a saved file is a straightforward process in DSPy. Here's how you can do it:

In [25]:
loaded_program = PlagiarismCoT()
loaded_program.load(path="BootstrapFewShotWithRandomSearch_program.json")

# 6.1 Deploying a DSPy Program

Deploying a DSPy program in a production environment can be challenging, as DSPy is primarily designed for research and experimentation at the moment. However, there are two effective methods to successfully deploy a DSPy program in production:

## 6.1.1 Using DSPy as a Library

This method involves saving, loading, and using your DSPy program directly:

1.  Save your DSPy program to a file using the `.save()` method.
2.  Load your DSPy program from the file using the `.load()` method.
3.  Use the loaded program to make predictions in your production environment.

## 6.1.2 Extracting Prompts and Calling Your LLM Directly

This approach involves:

1.  Extracting the optimized prompts from your DSPy program.
2.  Implementing these prompts in your production code.
3.  Calling your Language Model (LLM) directly with the extracted prompts.

Each method has its advantages and may be more suitable depending on your specific use case and production environment constraints.

In [26]:
# dummy call of your program
res = loaded_program(examples[0].code_sample_1, examples[0].code_sample_2)

# find last call in your llm
last_prompt = lm.inspect_history(n=1)
with open("inspect_history.txt", "w") as f:
    f.write(str(last_prompt))





[2025-03-14T18:17:54.932605]

System message:

Your input fields are:
1. `code_sample_1` (str): The first code sample to compare
2. `code_sample_2` (str): The second code sample to compare

Your output fields are:
1. `explanation` (str): Explanation or reason why the code samples are/ are not plagiarized
2. `plagiarized` (str): Yes/No indicating if code samples are plagiarized

All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## code_sample_1 ## ]]
{code_sample_1}

[[ ## code_sample_2 ## ]]
{code_sample_2}

[[ ## explanation ## ]]
{explanation}

[[ ## plagiarized ## ]]
{plagiarized}

[[ ## completed ## ]]

In adhering to this structure, your objective is: 
        Detect if two code samples are plagiarized. In plagiarized field answer only : Yes if the code samples are plagiarized, No otherwise. In explanation field add the reason why the code samples are/ are not plagiarized.


User message:

[[ ## code_sample_1 ## ]]
public clas